In [1]:
from __future__ import print_function, division
import os
import sys 

spark_home = os.environ['SPARK_HOME']
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.4-src.zip'))

from pyspark import SparkContext
from pyspark.streaming import StreamingContext

## Map vs Transform

In [ ]:
sc = SparkContext("local[2]", "NetworkWordCount")
ssc = StreamingContext(sc, batchDuration=5)


In [ ]:
mapper = sc.parallelize([('cat', 1),('dog', 2),('pig', 3)])
a = sc.parallelize([('cat', 1),('dog', 2),('pig', 3)])
a.join(mapper).collect()

In [ ]:
lines = ssc.socketTextStream("localhost", 9999)
words = lines.flatMap(lambda line: line.split(" ")).map(lambda x: (x, 1))
words.pprint()

In [ ]:
#mapping = words.map(lambda x: x.join(mapper))
mapping = words.transform(lambda x: x.join(mapper))
#mapping = words.join(mapper)

In [ ]:
mapping.pprint()

In [ ]:
ssc.start()
ssc.awaitTermination()

## updateStateByKey

In [ ]:
sc = SparkContext("local[2]", "NetworkWordCount")
ssc = StreamingContext(sc, batchDuration=5)
ssc.checkpoint("checkpoint")

In [ ]:
initialStateRDD = sc.parallelize([(u'hello', 1), (u'world', 1)])

def updateFunc(new_values, last_sum):
    return sum(new_values) + (last_sum or 0)

lines = ssc.socketTextStream("localhost", 9999)
running_counts = lines.flatMap(lambda line: line.split(" "))\
                          .map(lambda word: (word, 1))\
                          .updateStateByKey(updateFunc, initialRDD=initialStateRDD)

lines.pprint()
running_counts.pprint()

ssc.start()
ssc.awaitTermination()